In [ ]:
import pandas as pd
sheet_url='https://docs.google.com/spreadsheets/d/1MvIxVRBs4u6UFfvnu2khnU_gtrcoLUch5Tci-hcdkAk/edit#gid=635530870'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df = pd.read_csv(csv_export_url)
df

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,AND 1 = utl_inaddr.get_host_address ( ( SE...,1
...,...,...
37939,"org/?option = com_k2 <a href = ""http://corfopym",0
37940,com/?option = com_k2 <act> <![CDATA[procMemb...,0
37941,picsearch,0
37942,com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...,0


In [ ]:
sentence_col = df.drop(['Label'],axis='columns')
Y = df['Label']
sentence_col

,Sentence
0,NaN
1,""" or pg_sleep ( __TIME__ ) --"
2,create user name identified by pass123 tempora...
3,%29
4,AND 1 = utl_inaddr.get_host_address ( ( SE...
...,...
37939,"org/?option = com_k2 <a href = ""http://corfopym"
37940,com/?option = com_k2 <act> <![CDATA[procMemb...
37941,picsearch
37942,com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...


# Pre Processing

1) removed \n at the end of each line

2) replaced %20 by space #### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [ ]:
# preprocess sql data to have same format for all files

def clean_sqli_data(data):
    
    for i in range(len(data)):
        
        data[i]=data[i].replace('\n', '')
        data[i]=data[i].replace('%20', ' ')
        data[i]=data[i].replace('=', ' = ')
        data[i]=data[i].replace('((', ' (( ')
        data[i]=data[i].replace('))', ' )) ')
        data[i]=data[i].replace('(', ' ( ')
        data[i]=data[i].replace(')', ' ) ')
    
    return data

In [ ]:

# removing the row if "Sentence" column contains null  value
df.dropna(subset = ["Sentence"], inplace=True)


sql_lines=[]
labels=[]
# inserting dataframe data into list
for index, row in df.iterrows():
    sql_lines.append(row['Sentence'] )
    labels.append(row['Label'])
    #print(type(row['Sentence']))
    #print(row['Sentence'])


cleaned_data=clean_sqli_data(sql_lines)
cleaned_df = pd.DataFrame(list(zip(cleaned_data, labels)),columns =['Query', 'Label'])
cleaned_df

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,%29,1
3,AND 1 = utl_inaddr.get_host_address ( ...,1
4,select * from users where id = '1' or @ @1 ...,1
...,...,...
37919,"org/?option = com_k2 <a href = ""http://co...",0
37920,com/?option = com_k2 <act> <![CDATA[procMe...,0
37921,picsearch,0
37922,com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...,0


# **Feature Extraction**

In [ ]:
#  Detect select/update/insert/delete keyword at the initial part of the query

def isFetchCommandType(query):

    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'select' ):
        return 1
    else:
        return 0

def isUpdateCommandType(query):
    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'update' ):
        return 1
    else:
        return 0
      
def isInsertCommandType(query):
    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'insert' ):
        return 1
    else:
        return 0

def isDeleteCommandType(query):
    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'delete' ):
        return 1
    else:
        return 0


print(isFetchCommandType("INSERT into(223,'sajid','2343113')") )
print(isUpdateCommandType("DELETE into(223,'brishty','2343113')") )
print(isInsertCommandType("DELETE into(223,'sajid','2343113')") )
print(isDeleteCommandType("DELETE into(223,'brishty','2343113')") )

0
0
0
1


In [ ]:
# Detect 'order by' keyword in the query

def detectSortClause(query):
    keyword='order by'
    if(keyword in query.lower()):
        return 1
    else:
        return 0
    
print(detectSortClause("-5021'+ ( select yadq where 4285 = 4285 Order By 1#"))

1


In [ ]:
  # Detect group by keyword
  def detectGroupClause(query):
      keyword= 'group by'  
      if(keyword in query.lower()):
          return 1
      else:
          return 0

print(detectGroupClause("-5021'+ ( select yadq where 4285 = 4285 Group By 1#"))

1


In [ ]:
# Detect over keyword 

def detectWindowClause(query):
    keyword='over'
    if(keyword in query.lower()):
        return 1
    else:
        return 0

print(detectWindowClause("-5021'+ ( select yadq where 4285 = 4285 order by 1#"))

0


In [ ]:
#hasRecursive: Indicator if SQL is specified WITH RECURSIVE clause
def detectWithRecursive(query):
    keyword='with recursive'
    if(keyword in query.lower()):
        return 1
    else:
        return 0
print(detectWithRecursive('With Recursive parent_of(name, parent) AS (SELECT name, mom FROM family UNION SELECT name, dad FROM family);'))


1


In [ ]:
#hasDistinctOn: Indicator if distinct clause is from DISTINCT ON
def detecthasDistinctOn(query):
    keyword= 'distinct on'
    if(keyword in query.lower()):
        return 1
    else:
        return 0

print(detecthasDistinctOn('SELECT * FROM (SELECT DISTINCT ON(address_id) * FROM purchases WHERE product_id = 1 ORDER BY address_id, purchased_at DESC) t ORDER BY purchased_at DESC;'))
print(detecthasDistinctOn('SELECT * FROM First JOIN SELECT * FROM Second;')  )


1
0


In [ ]:
#utilityStmt: Indicator if the SQL is DECLARE CURSOR or a non-optimizable statement

def utilityStmt(query):
    keyword1='declare'
    keyword2='cursor'
    if(keyword1 in query.lower() and keyword2 in query.lower()):
      return 1
    else:
      return 0

print(utilityStmt('EXEC SQL DECLARE C3 CURSOR WITH HOLD FOR SELECT * FROM EMPLOYEE FOR UPDATE OF WORKDEPT, PHONENO, JOB, EDLEVEL, SALARY;'))
print(utilityStmt('SELECT * FROM First JOIN SELECT * FROM Second;')  )


1
0


operators list in this link: https://www.w3schools.com/sql/sql_operators.asp

In [ ]:
#jointree/quals/args*/boolop* : Qualification type BOOLEXPR, OPEXPR and boolean operation and, or, not

def hasLogicalOperator(query):
      keywords=['and','or','not','all','any','between','exists','in','like','some']
      for key in keywords:
        if(key in query.lower()):
          return 1
        else:
          return 0

print(hasLogicalOperator('SELECT column-names FROM table-name WHERE condition1 AND condition2;'))
print(hasLogicalOperator('UPDATE table-name SET column-name = value WHERE condition1 OR condition2;'))
print(hasLogicalOperator('DELETE table-name WHERE NOT condition;'))

1
0
0


In [ ]:


def hasArithmaticOperator(query):
    keywords=['+','-','*','/','%']
   
    for key in keywords:
        if(key in query):
            return 1
        
    return 0


print(hasArithmaticOperator("SELECT city, SUM(order_amount) total_order_amount FROM sum GROUP BY city"))

0


In [ ]:

def hasComparisonOperator(query):
    keywords=['=','>','<','>=','<=','<>']
    
    for key in keywords:
        if(key in query):
            return 1
        
    return 0

print(hasComparisonOperator('SELECT * FROM Products WHERE Price = 18;' ) )

1


In [ ]:

def hasCompoundOperator(query):
    keywords=['+=','-=',"*=",'/=','%=','&=','^-=','|*=']
   
    for key in keywords:
        if(key in query):
            return 1
    return 0


print(hasCompoundOperator('SELECT * FROM Products WHERE Price = 18;' ))

0


**List of window functions**: https://www.sqlshack.com/use-window-functions-sql-server/

In [ ]:
# detect window function in the query RANK(), DENSE_RANK(), ROW_NUMBER(), NTILE(),SUM(), MAX(), MIN(), AVG(). COUNT(),LAG(), LEAD(), FIRST_VALUE(), LAST_VALUE(),
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def hasWindowFunc(query):
    window_func_list=['sum','avg','max','min','count','rank','dense_rank','row_number','nitle','lag','lead','first_value','last_value']
    tokenized_list = word_tokenize(query.lower())
   
    #print(tokenized_list)
    for window_func in window_func_list:
        #print(window_func)
        for i in range(len(tokenized_list)):
            if(window_func == tokenized_list[i]):
                if(i+1 < len(tokenized_list)):
                      if( tokenized_list[i+1] == '('):
                          return 1
                      else:
                          return 0
    return 0



hasWindowFunc("SELECT city, SUM(order_amount) total_order_amount FROM sum GROUP BY city")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


1

In [ ]:
# hasAggs: Indicator if SQL has aggregates (sum(),avg(),max(),min(),count())
def hasAggs(query):
    Agg_func_list=['sum','avg','max','min','count']
    tokenized_list = word_tokenize(query.lower())
    
    #print(tokenized_list)
    for Agg_func in Agg_func_list:
        #print(window_func)
        for i in range(len(tokenized_list)):
            if(Agg_func == tokenized_list[i]):
                if( tokenized_list[i+1] == '('):
                    return 1
                else:
                    return 0
    return 0

    
print(hasAggs("-5021'+ avg( select yadq where 4285 = 4285 order by 1#"))
print(utilityStmt('SELECT * FROM First JOIN SELECT * FROM Second;')  )
print(hasAggs("-5021'+ max( select yadq where 4285 = 4285 order by 1#"))
print(hasAggs("-5021'+ min( select yadq where 4285 = 4285 order by 1#"))
print(hasAggs("-5021'+ count( select yadq where 4285 = 4285 order by 1#"))

1
0
1
1
1


In [ ]:
#detect set operations like union,union all, except, intersect

def hasSetOperation(query):
    set_list=['union','union all','except','intersect']
    tokenized_list = word_tokenize(query.lower())
   
    #print(tokenized_list)
    for set_op in set_list:
        #print(window_func)
        for i in range(len(tokenized_list)):
            if(set_op in tokenized_list):
                return 1
            
    return 0

print(hasSetOperation('SELECT * FROM First JOIN SELECT * FROM Second;'))

0


In [ ]:
#cteList---> getNumberOfWithClause() 
# detect number of with clasue 

def getNumberOfWithClause(query) :
    keyword='with'
    
    tokenized_list = word_tokenize(query.lower())
    print(tokenized_list)
    cnt=0
    for word in tokenized_list:
        if(keyword == word ):
            cnt +=1
    
    return cnt    




print(getNumberOfWithClause( ' WITH totalSalary(Airline, total) as (SELECT Airline, sum(Salary) FROM Pilot GROUP BY Airline),airlineAverage(avgSalary) as (SELECT avg(Salary) FROM Pilot )  SELECT Airline FROM totalSalary, airlineAverage WHERE totalSalary.total >= airlineAverage.avgSalary;'))

['with', 'totalsalary', '(', 'airline', ',', 'total', ')', 'as', '(', 'select', 'airline', ',', 'sum', '(', 'salary', ')', 'from', 'pilot', 'group', 'by', 'airline', ')', ',', 'airlineaverage', '(', 'avgsalary', ')', 'as', '(', 'select', 'avg', '(', 'salary', ')', 'from', 'pilot', ')', 'select', 'airline', 'from', 'totalsalary', ',', 'airlineaverage', 'where', 'totalsalary.total', '>', '=', 'airlineaverage.avgsalary', ';']
1


In [ ]:
# detect sub query in the query

def detectSubQuery(query):
    commands = ['select','update','insert','delete']
    tokenized_list = word_tokenize(query.lower())
    cnt=0
    for command in commands:
        if(command in tokenized_list):
            cnt+=1
    if(cnt>1):
        return 1
    else:
        return 0


print(detectSubQuery('UPDATE CUSTOMERS SET SALARY = SALARY * 0.25 WHERE AGE IN ( SELECT AGE FROM CUSTOMERS_BKP WHERE AGE >= 27 );'))

print( detectSubQuery('INSERT INTO CUSTOMERS_BKP SELECT * FROM CUSTOMERS WHERE ID IN (SELECT ID FROM CUSTOMERS) ;') )

1
1


In [ ]:
# detect string constant type 


def hasStringConstnt(query):
    ck=0
    new_query=query.lower()
    
    for i in range(len(new_query)):
        
        if(new_query[i] == '\''):
            ck=1
            break
            
    if(ck==1 ):
        return 1
    else:
        return 0

print(hasStringConstnt("SELECT * FROM Customers WHERE Country='Mexico' and city='\t';") )

1


Constants:  https://ecomputernotes.com/what-is-c/types-and-variables/what-is-constants-type-of-constant

In [ ]:
# detect backslash character constants

def hasEscapeSeq(query):
    escape_chars=['\a',
                  '\b',
                  '\f',
                  '\n',
                  '\r',
                  '\t',
                  '\v',
                  '\'',
                  "\''",
                  '\?',
                  '\\',
                  '\0']
    for chars in escape_chars:
        if(chars in  query.lower()):
            return 1
        
    return 0    


print(hasEscapeSeq("SELECT * FROM Customers WHERE Country='Mexico' and city='';") )


1


In [ ]:
# detect numbers in query
from sklearn.feature_extraction.text import CountVectorizer

def detectNumber(query):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([query])
    print(vectorizer.get_feature_names())
    tokenized_list=vectorizer.get_feature_names()
    for word in tokenized_list:
        ck=0
        try:
          num = int(word)
          ck=1 
        except:
          try:
            num = float(word)
            ck=2
          except:
            ck=3
        if(ck == 1 or ck == 2):
          return 1 #number
        else:
          return 0 #non-number
        
      
    


print(detectNumber("SELECT * FROM Customers WHERE AGE==20 "))

['20', 'age', 'customers', 'from', 'select', 'where']
1


# Encapsulation of feature extraction functions inside a class

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

class SqlFeatureExtraction:

    def __init__(self):
        print("SQL feature extraction object created")

    def extractFeatures(self,query):
        features=[]
        features.append( isFetchCommandType(query) )
        features.append( isUpdateCommandType(query) )
        features.append( isInsertCommandType(query) )
        features.append( isDeleteCommandType(query) )
        features.append( detectSortClause(query) )
        features.append( detectGroupClause(query) )
        features.append( detectWindowClause(query) )
        features.append( detectWithRecursive(query) )
        features.append( detecthasDistinctOn(query) )
        features.append( utilityStmt(query) )
        features.append(hasLogicalOperator(query) )
        features.append( hasArithmaticOperator(query)  )
        features.append( hasComparisonOperator(query)  )
        features.append( hasCompoundOperator(query) )
        features.append( hasWindowFunc(query) )
        features.append( hasAggs(query)  )
        features.append( hasSetOperation(query) )  
        features.append(getNumberOfWithClause(query))
        features.append( detectSubQuery(query) )
        features.append( hasStringConstnt(query) )
        features.append( hasEscapeSeq(query) )
        features.append( detectNumber(query) )
        return features
    
    #  Detect select/update/insert/delete keyword at the initial part of the query
    # f1
    def isFetchCommandType(query):

        intial_keyword = query.split(' ')[0].lower()
        if( intial_keyword == 'select' ):
            return 1
        else:
            return 0

    #f2
    def isUpdateCommandType(query):
        intial_keyword = query.split(' ')[0].lower()
        if( intial_keyword == 'update' ):
            return 1
        else:
            return 0
    #f3   
    def isInsertCommandType(query):
        intial_keyword = query.split(' ')[0].lower()
        if( intial_keyword == 'insert' ):
            return 1
        else:
            return 0
    #f4
    def isDeleteCommandType(query):
        intial_keyword = query.split(' ')[0].lower()
        if( intial_keyword == 'delete' ):
            return 1
        else:
            return 0


    # Detect 'order by' keyword in the query
    #f5
    def detectSortClause(query):
        keyword='order by'
        if(keyword in query.lower()):
            return 1
        else:
            return 0

    # Detect group by keyword
    def detectGroupClause(query):
          keyword= 'group by'  
          if(keyword in query.lower()):
              return 1
          else:
              return 0


    # Detect over keyword 

    def detectWindowClause(query):
        keyword='over'
        if(keyword in query.lower()):
            return 1
        else:
            return 0


    #hasRecursive: Indicator if SQL is specified WITH RECURSIVE clause
    def detectWithRecursive(query):
        keyword='with recursive'
        if(keyword in query.lower()):
            return 1
        else:
            return 0


    #hasDistinctOn: Indicator if distinct clause is from DISTINCT ON
    def detecthasDistinctOn(query):
        keyword= 'distinct on'
        if(keyword in query.lower()):
            return 1
        else:
            return 0

    #utilityStmt: Indicator if the SQL is DECLARE CURSOR or a non-optimizable statement

    def utilityStmt(query):
        keyword1='declare'
        keyword2='cursor'
        if(keyword1 in query.lower() and keyword2 in query.lower()):
          return 1
        else:
          return 0

    #jointree/quals/args*/boolop* : Qualification type BOOLEXPR, OPEXPR and boolean operation and, or, not

    def hasLogicalOperator(query):
          keywords=['and','or','not','all','any','between','exists','in','like','some']
          for key in keywords:
            if(key in query.lower()):
              return 1
            else:
              return 0

    def hasArithmaticOperator(query):
        keywords=['+','-','*','/','%']
      
        for key in keywords:
            if(key in query):
                return 1
            
        return 0

    def hasComparisonOperator(query):
        keywords=['=','>','<','>=','<=','<>']
        
        for key in keywords:
            if(key in query):
                return 1
            
        return 0


    def hasCompoundOperator(query):
        keywords=['+=','-=',"*=",'/=','%=','&=','^-=','|*=']
      
        for key in keywords:
            if(key in query):
                return 1
        return 0



    # detect window function in the query RANK(), DENSE_RANK(), ROW_NUMBER(), NTILE(),SUM(), MAX(), MIN(), AVG(). COUNT(),LAG(), LEAD(), FIRST_VALUE(), LAST_VALUE(),


    def hasWindowFunc(query):
        window_func_list=['sum','avg','max','min','count','rank','dense_rank','row_number','nitle','lag','lead','first_value','last_value']
        tokenized_list = word_tokenize(query.lower())
      
        #print(tokenized_list)
        for window_func in window_func_list:
            #print(window_func)
            for i in range(len(tokenized_list)):
                if(window_func == tokenized_list[i]):
                    if(i+1 < len(tokenized_list)):
                        if( tokenized_list[i+1] == '('):
                            return 1
                        else:
                            return 0
        return 0


    # hasAggs: Indicator if SQL has aggregates (sum(),avg(),max(),min(),count())
    def hasAggs(query):
        Agg_func_list=['sum','avg','max','min','count']
        tokenized_list = word_tokenize(query.lower())
        
        #print(tokenized_list)
        for Agg_func in Agg_func_list:
            #print(window_func)
            for i in range(len(tokenized_list)):
                if(Agg_func == tokenized_list[i]):
                    if( tokenized_list[i+1] == '('):
                        return 1
                    else:
                        return 0
        return 0


    #detect set operations like union,union all, except, intersect

    def hasSetOperation(query):
        set_list=['union','union all','except','intersect']
        tokenized_list = word_tokenize(query.lower())
      
        #print(tokenized_list)
        for set_op in set_list:
            #print(window_func)
            for i in range(len(tokenized_list)):
                if(set_op in tokenized_list):
                    return 1
                
        return 0


    #cteList---> getNumberOfWithClause() 
    # detect number of with clasue 

    def getNumberOfWithClause(query) :
        keyword='with'
        
        tokenized_list = word_tokenize(query.lower())
        #print(tokenized_list)
        cnt=0
        for word in tokenized_list:
            if(keyword == word ):
                cnt +=1
        
        return cnt



    # detect sub query in the query

    def detectSubQuery(query):
        commands = ['select','update','insert','delete']
        tokenized_list = word_tokenize(query.lower())
        cnt=0
        for command in commands:
            if(command in tokenized_list):
                cnt+=1
        if(cnt>1):
            return 1
        else:
            return 0


    # detect string constant type 

    def hasStringConstnt(query):
        ck=0
        new_query=query.lower()
        
        for i in range(len(new_query)):
            
            if(new_query[i] == '\''):
                ck=1
                break
                
        if(ck==1 ):
            return 1
        else:
            return 0



    # detect backslash character constants

    def hasEscapeSeq(query):
        escape_chars=['\a',
                      '\b',
                      '\f',
                      '\n',
                      '\r',
                      '\t',
                      '\v',
                      '\'',
                      "\''",
                      '\?',
                      '\\',
                      '\0']
        for chars in escape_chars:
            if(chars in  query.lower()):
                return 1
            
        return 0    


    # detect numbers in query


    def detectNumber(query):
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform([query])
        #print(vectorizer.get_feature_names())
        tokenized_list=vectorizer.get_feature_names()
        for word in tokenized_list:
            ck=0
            try:
              num = int(word)
              ck=1 
            except:
              try:
                num = float(word)
                ck=2
              except:
                ck=3
            if(ck == 1 or ck == 2):
              return 1 #number
            else:
              return 0 #non-number
            
      








[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
column_names=['FetchCommandType','UpdateCommandType','InsertCommandType',
             'DeleteCommandType','detectSortClause','detectGroupByClause',
             'detectWindowClause','detectWithRecursive','detectDistinctOn',
             'detectDeclareCursor','detectLogicalOperator',
             'detectArithmaticOperator','detectComparisonOperator','hasCompoundOperator',
             'detectWindowFunc','detectAggregates','detectSetOperation',
             'detectNumberOfWithClause','detectSubQuery',
             'detectStringConstant','detectBackslash','detectNumber','Label']

len(column_names)

23

In [ ]:


feature_val_list=[]



for index, row in cleaned_df.iterrows():
    obj = SqlFeatureExtraction()
    try:
      feature_val = obj.extractFeatures(row['Query'])
      feature_val.append(row['Label'])
      feature_val_list.append(feature_val)
    except ValueError:
      print("value Error: "+str(ValueError.args))


feature_table = pd.DataFrame(feature_val_list,columns=column_names)



Streaming output truncated to the last 5000 lines.
['athletes', 'assistants', 'seemed', 'satisfied', 'smooth', 'well-organised', 'running', 'event']
['assistants', 'athletes', 'event', 'organised', 'running', 'satisfied', 'seemed', 'smooth', 'well']
SQL feature extraction object created
['rbs', 'suspends', 'two', 'forex', 'traders', 'royal', 'bank', 'scotland', 'suspended', 'two', 'traders', 'foreign', 'exchange', 'division', 'according', 'two', 'people', 'familiar', 'situation', ',', 'another', 'sign', 'global', 'probe', 'regulators', 'suspected', 'manipulation', 'currency', 'market', 'rapidly', 'gaining', 'traction']
['according', 'another', 'bank', 'currency', 'division', 'exchange', 'familiar', 'foreign', 'forex', 'gaining', 'global', 'manipulation', 'market', 'people', 'probe', 'rapidly', 'rbs', 'regulators', 'royal', 'scotland', 'sign', 'situation', 'suspected', 'suspended', 'suspends', 'traction', 'traders', 'two']
SQL feature extraction object created
['some', 'world', '&', 'ap

# Write feature table to CSV

In [ ]:
feature_table.to_csv("sql_preprocessed_table.csv")

# Spliting the dataframe 

In [ ]:
X = feature_table.drop(['Label'],axis='columns')
Y = feature_table['Label']

In [ ]:
X

,FetchCommandType,UpdateCommandType,InsertCommandType,DeleteCommandType,detectSortClause,detectGroupByClause,detectWindowClause,detectWithRecursive,detectDistinctOn,detectDeclareCursor,detectLogicalOperator,detectArithmaticOperator,detectComparisonOperator,hasCompoundOperator,detectWindowFunc,detectAggregates,detectSetOperation,detectNumberOfWithClause,detectSubQuery,detectStringConstant,detectBackslash,detectNumber
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37704,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
37705,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1
37706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37707,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0


In [ ]:
Y

0        1
1        1
2        1
3        1
4        1
        ..
37704    0
37705    0
37706    0
37707    0
37708    0
Name: Label, Length: 37709, dtype: int64

# Installing Pycaret and sklearn-genetic

In [ ]:
pip install pycaret

     |████████████████████████████████| 266kB 6.5MB/s 
     |████████████████████████████████| 112kB 10.7MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 276kB 9.3MB/s 
     |████████████████████████████████| 6.8MB 15.1MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 14.2MB 199kB/s 
     |████████████████████████████████| 1.7MB 25.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.4MB 16.7MB/s 
     |████████████████████████████████| 2.0MB 36.4MB/s 
     |████████████████████████████████| 256kB 56.7MB/s 
     |████████████████████████████████| 174kB 52.1MB/s 
     |████████████████████████████████| 1.2MB 39.6MB/s 
     |████████████████████████████████| 153kB 44.7MB/s 
     |████████████████████████████████| 1.1MB 26.9MB/

In [ ]:
pip install sklearn-genetic

     |████████████████████████████████| 163kB 5.1MB/s 


In [ ]:
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from genetic_selection import GeneticSelectionCV

mcc = make_scorer(matthews_corrcoef)
estimator = LogisticRegression(solver = "liblinear", C = 6, tol = 1, fit_intercept = True)

from sklearn.model_selection import *
report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 
rkf = RepeatedStratifiedKFold(n_repeats = 2, n_splits = 10)
for i in range(2,22):
  
    selector = GeneticSelectionCV(estimator,
                                cv = rkf,
                                verbose = 0,
                                scoring = mcc,
                                max_features = i,
                                n_population = 50,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                #tournament_size = 3,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  
    selector = selector.fit(X, Y)
    genfeats = X.columns[selector.support_]
    genfeats = list(genfeats)
    print("Chosen Feats:  ", genfeats)

cv_score = selector.generation_scores_[-1]
nofeats.append(len(genfeats)) 
chosen_feats.append(genfeats) 
cvscore.append(cv_score)





report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore






/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in d

Chosen Feats:   ['detectComparisonOperator']


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in d

Chosen Feats:   ['detectArithmaticOperator', 'detectComparisonOperator', 'detectStringConstant']


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in d

...

Chosen Feats:   ['detectGroupByClause', 'detectWindowClause', 'detectArithmaticOperator', 'detectComparisonOperator', 'detectSetOperation', 'detectNumberOfWithClause', 'detectSubQuery', 'detectStringConstant']


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in d

Chosen Feats:   ['FetchCommandType', 'UpdateCommandType', 'DeleteCommandType', 'detectGroupByClause', 'detectWindowClause', 'detectWithRecursive', 'detectDistinctOn', 'detectDeclareCursor', 'detectLogicalOperator', 'detectArithmaticOperator', 'detectComparisonOperator', 'detectAggregates', 'detectSetOperation', 'detectNumberOfWithClause', 'detectSubQuery', 'detectStringConstant', 'detectNumber']


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  >>> zero_one_loss(y_true, y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in d

Chosen Feats:   ['FetchCommandType', 'UpdateCommandType', 'detectGroupByClause', 'detectWindowClause', 'detectWithRecursive', 'detectDistinctOn', 'detectDeclareCursor', 'detectLogicalOperator', 'detectArithmaticOperator', 'detectComparisonOperator', 'hasCompoundOperator', 'detectWindowFunc', 'detectSetOperation', 'detectSubQuery', 'detectStringConstant', 'detectNumber']


In [ ]:
report["Scores"] = np.round(report["Scores"], 3)
report.sort_values(by = "Scores", ascending = False, inplace = False)
#report.index
ga_feats = report.iloc[0]["Chosen Feats"]

used_feats =  ga_feats.copy()
used_feats.append('ClassLabel')

print(report)
print(ga_feats)
